In [83]:
import cv2
import numpy as np

In [84]:
def compute_gradient_magnitude(image):
    gradient_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    gradient_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    
    gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
    
    return gradient_magnitude

In [85]:
def find_vertical_seam(magnitude):
    rows, cols = magnitude.shape
    seam = []
    
    mag_copy = np.copy(magnitude)
    
    # find the cumulative magnitude values to decide a starting point for backtracking
    
    for i in range(1, rows):
        for j in range(cols):
            left = mag_copy[i - 1, j - 1] if j > 0 else np.inf
            center = mag_copy[i - 1, j]
            right = mag_copy[i - 1, j + 1] if j < cols-1 else np.inf
            
            mag_copy[i, j] += min(left, right, center)
            
    # choose the index with the smallest cumulative magnitude as a starting point for the finding a seam
    min_mag_index = np.argmin(mag_copy[-1, :])
    seam.append(min_mag_index)
    
    # Backtrack to find the entire seam - start on the last row
    for i in range(rows - 2, -1, -1):
        if min_mag_index == 0:
            # on first col so only two options
            indices = [min_mag_index, min_mag_index + 1]
        elif min_mag_index == cols - 1:
            # on last col so only two options
            indices = [min_mag_index - 1, min_mag_index]
        else:
            # the three x-value options to choose from when traversing up
            indices = [min_mag_index - 1, min_mag_index, min_mag_index + 1]
        # choose the min between the 3 option magnitudes
        min_mag_index = indices[np.argmin(mag_copy[i, indices])]
        # add x index to seam (value is assumed)
        seam.insert(0, min_mag_index)
    
    return seam



In [96]:
def remove_vertical_seam(img, seam):
    rows, cols = img.shape
    new_img = np.zeros((rows, cols - 1), dtype=np.uint8)
    
    for i in range(rows):
        j_seam = seam[i]
        new_img[i] = np.delete(img[i], j_seam, axis=0)
    
    return new_img

In [99]:
def highlight_seam(img, seam):
    new_img = np.copy(img)
    rows, cols = img.shape
    for i in range(rows):
        new_img[i][seam[i]] = 255
    return new_img

In [97]:
filepath = 'assets/test.jpg'
img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

if img is None:
    print(f"Error: Unable to read the image at '{''}'")

In [103]:
# loop for how many seams to remove
for _ in range(10):
    gradient_magnitude = compute_gradient_magnitude(img)
    seam = find_vertical_seam(gradient_magnitude)
    img = remove_vertical_seam(img, seam)

# h = highlight_seam(img, seam)
# cv2.imshow("H_Image", h)

cv2.namedWindow("Original_Image", cv2.WINDOW_NORMAL)
cv2.namedWindow("Changed_Image", cv2.WINDOW_NORMAL)

cv2.imshow("Original_Image", cv2.imread(filepath))
cv2.imshow("Changed_Image", img)

cv2.waitKey(0)
cv2.destroyAllWindows()